In [16]:
# Por ahora, se corre solamente "Configuraciones generales" y "Ajustes por inflación".

## A. Configuraciones generales.

In [17]:
#1. Librerías.
%run "../librerias.ipynb"

In [18]:
#2. Constantes.
%run "../constantes.ipynb"

mes_train = mes_train_all_menos_1
mes_test = mes_test
dataset_output = dataset_file_preprocesado_all_1

In [19]:
#3. Funciones.
%run "../funciones.ipynb"

In [ ]:
#4. Lectura de datos.
data = pd.read_csv(dataset_file_clase_ternaria)

## B. Data Quality.

#### 1. Valores nulos.

In [ ]:
#a. Evaluamos.
train_data = data[data['foto_mes'].isin(mes_train)]
score_data = data[data['foto_mes'] == mes_test]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted

#### 2. Valores 0.

In [ ]:
#a. Evaluamos.
train_zero_percentage = (train_data == 0).mean() * 100
score_zero_percentage = (score_data == 0).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Zero Percentage': train_zero_percentage, 'Score Zero Percentage': score_zero_percentage})

comparison_df_zero['diff_zero_percentage'] = (comparison_df_zero['Score Zero Percentage'] - comparison_df_zero['Train Zero Percentage']).abs()
diff_zero_percentage_sorted = comparison_df_zero.sort_values('diff_zero_percentage',ascending=False)

In [ ]:
#b. Decidimos eliminar aquellas columnas con un gran cambio en la cantidad de 0 en el mes test.
cols_eliminar = list(diff_zero_percentage_sorted[(diff_zero_percentage_sorted["diff_zero_percentage"] > 5)&(diff_zero_percentage_sorted["Score Zero Percentage"]>diff_zero_percentage_sorted["Train Zero Percentage"])].index)
data.drop(cols_eliminar,axis=1,inplace=True)

train_data.drop(cols_eliminar,axis=1,inplace=True)
score_data.drop(cols_eliminar,axis=1,inplace=True)

#### 3. Valores <0.

In [ ]:
#a. Evaluamos.
train_negative_percentage = (train_data == -1).mean() * 100
score_negative_percentage = (score_data == -1).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Negative Percentage': train_negative_percentage, 'Score Negative Percentage': score_negative_percentage})

comparison_df_zero['diff_negative_percentage'] = (comparison_df_zero['Score Negative Percentage'] - comparison_df_zero['Train Negative Percentage']).abs()
diff_negative_percentage_sorted = comparison_df_zero.sort_values('diff_negative_percentage',ascending=False)

diff_negative_percentage_sorted

#### 4. Valores duplicados.

In [ ]:
#a. Verifico.
print("Train tiene {} valores duplicados".format(train_data.duplicated().sum()))
print("Test tiene {} valores duplicados".format(score_data.duplicated().sum()))

## C. Data Drifting.

In [ ]:
#1. Aplicamos PSI a casi todas las variables, excepto las de no interés.
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes','numero_de_cliente','clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)

In [ ]:
#2. Observamos variables con PSI >= 0.1 (casos a atender).
psi_df[psi_df["psi"] > 0.1]["feature"]

## D. Ajustes por inflación.

In [ ]:
#1. Defino los campos que voy a ajustar por inflación.
campos_monetarios = [col for col in data.columns if col.startswith(('m', 'Visa_m', 'Master_m', 'vm_m'))]

In [ ]:
#2. Le aplico la inflación a todas ellas.
data = drift_deflacion(data, campos_monetarios, tb_indices)

## E. Exportación.

In [ ]:
#1. Exportación propiamente dicha.
data.to_csv(dataset_output,index=False)